In [1]:
import tensorflow as tf

import keras_tuner as kt

from tensorflow.keras import models, layers, datasets

In [2]:
(train_image, train_label), (test_image, test_label) = datasets.fashion_mnist.load_data()

In [3]:
def rescale(image):
  """
  Helper Function
  """
  image = tf.cast(image, tf.float32) / 255.0

  return image

train_image = rescale(train_image)

test_image = rescale(test_image)

In [4]:
import warnings; warnings.filterwarnings('ignore')

input_shape = (28, 28)

model = models.Sequential()

model.add(layers.Flatten(input_shape=input_shape))

model.add(layers.Dense(16, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [5]:
loss = 'sparse_categorical_crossentropy'

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [6]:
loop = 10

hist = model.fit(train_image, train_label, epochs=loop, validation_data=(test_image, test_label))

In [7]:
model.evaluate(train_image, train_label, verbose=False)

[0.3282566964626312, 0.882266640663147]

In [8]:
model.evaluate(test_image, test_label, verbose=False)

[0.4059363305568695, 0.858299970626831]

In [9]:
def builder(hp):
  """
  Model Builder
  """
  model = models.Sequential()

  model.add(layers.Flatten(input_shape=input_shape))

  units = hp.Int('units', min_value=16, max_value=64, step=16)

  model.add(layers.Dense(units, activation='relu'))

  model.add(layers.Dense(10, activation='softmax'))

  model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

  return model

In [10]:
dir = 'Tuner'

target = 'val_accuracy'

project = 'Belajar Keras Tuner'

tuner = kt.Hyperband(builder, objective=target, max_epochs=loop, directory=dir, project_name=project)

Reloading Tuner from Tuner/Belajar Keras Tuner/tuner0.json


In [11]:
tuner.search(train_image, train_label, validation_data=(test_image, test_label), epochs=loop)

In [12]:
optima = tuner.get_best_hyperparameters()[0]

optima.get('units')

64

In [13]:
resulta = tuner.get_best_models()[0]

hist = resulta.fit(train_image, train_label, validation_data=(test_image, test_label), epochs=loop)

In [14]:
resulta.evaluate(train_image, train_label)

[0.23577918112277985, 0.9132833480834961]

In [15]:
resulta.evaluate(test_image, test_label)

[0.347107857465744, 0.8809999823570251]